### Task 1. 

#### MapReduce

Use MapReduce to calculate the average housing price in each zip code and filter out records in which gross sqft is less than 500 and sale price is less than 100000. Round the average housing price to 2 decimals, and sort the output by housing price ascending.

In [1]:
!wget -O housingSalseSample.csv https://github.com/CUSP2022ADS/Data/raw/main/housingSalseSample.csv

--2022-02-14 03:17:06--  https://github.com/CUSP2022ADS/Data/raw/main/housingSalseSample.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CUSP2022ADS/Data/main/housingSalseSample.csv [following]
--2022-02-14 03:17:07--  https://raw.githubusercontent.com/CUSP2022ADS/Data/main/housingSalseSample.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6167298 (5.9M) [text/plain]
Saving to: ‘housingSalseSample.csv’

housingSalseSample. 100%[===================>]   5.88M  --.-KB/s    in 0.06s   

2022-02-14 03:17:07 (105 MB/s) - ‘housingSalseSample.csv’ saved [6167298/6167298]



In [2]:
# code here
import pandas as pd
import csv
from functools import reduce
from multiprocessing import Pool

In [3]:
HP = pd.read_csv('housingSalseSample.csv')
HP[:3]

,BOROUGH,BLOCK,LOT,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,1170,1010,17 CONDOPS,"200 WEST 79TH STREET, 7P",10024.0,0.0,1975.0,551000,2011-07-28
1,4,1275,45,10 COOPS - ELEVATOR APARTMENTS,"35-35 75TH STREET, 304",11372.0,0.0,1941.0,338000,2011-05-20
2,3,5746,58,03 THREE FAMILY DWELLINGS,1249 65TH STREET,11219.0,3861.0,1910.0,1250000,2014-12-04


In [4]:
def returnInfo(record): #extract necessary fields from a data record (raw)
    return(record['ZIP CODE'],record['SALE PRICE'],record['GROSS SQUARE FEET'])
    
def filtersqp(record):
    price, sqft = record[1], record[2]
    if sqft > '500' and price > '100000':
        return True
    else:
        return False

def pricelist(result, record):
    zipcode, price = record[0], float(record[1])
    result[zipcode] = result.get(zipcode, []) + [price]
    return result

def averageprice(record):
    zipcode, price = record[0], record[1]
    return [zipcode, "{:.2f}".format(float(sum(price))/float(len(price)))]

In [5]:
%time
with open('housingSalseSample.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output = list(map(averageprice,list(reduce(pricelist,list(filter(filtersqp, list(map(returnInfo,reader)))),{}).items())))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


Breakdown

In [6]:
with open('housingSalseSample.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output1 = list(map(returnInfo,reader))
output1[:3]

[('10024.0', '551000', '0.0'),
 ('11372.0', '338000', '0.0'),
 ('11219.0', '1250000', '3861.0')]

In [7]:
output2 = list(filter(filtersqp, output1))
output2[:3]

[('11214.0', '2000000', '5054.0'),
 ('10024.0', '4550000', '5301.0'),
 ('11215.0', '10775000', '7660.0')]

In [8]:
output3 = reduce(pricelist, output2,{})

In [9]:
output4 = list(map(averageprice,list(output3.items())))

In [10]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['zipcode','price']).to_csv('output1.csv',index=False)

grading(output)

In [11]:
import pandas
import numpy as np
result = pd.DataFrame(output, columns=['zipcode','price']).reset_index(inplace = False)
result['price'] = result['price'].apply(pd.to_numeric)
result = result[['zipcode','price']]
result = result.sort_values(by = 'price', ascending = True)
result

,zipcode,price
147,11369.0,15000.00
125,11428.0,291845.00
18,11434.0,294485.18
43,11693.0,298857.14
32,10306.0,309288.43
...,...,...
112,10036.0,22546012.50
73,10010.0,26826000.00
30,10018.0,28609507.60
121,10001.0,49613816.00


In [12]:
result.describe()

,price
count,1.730000e+02
mean,3.780890e+06
std,7.389601e+06
min,1.500000e+04
25%,6.363122e+05
50%,1.431537e+06
75%,3.999300e+06
max,6.735514e+07


### Task 2. 
#### multiprocessing and Chunk

In [13]:
!wget -O ADSSession2Task2Data.csv https://raw.githubusercontent.com/CUSP2022ADS/Data/main/ADSSession2Task2Data.csv

--2022-02-14 03:17:08--  https://raw.githubusercontent.com/CUSP2022ADS/Data/main/ADSSession2Task2Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279369 (273K) [text/plain]
Saving to: ‘ADSSession2Task2Data.csv’

ADSSession2Task2Dat 100%[===================>] 272.82K  --.-KB/s    in 0.02s   

2022-02-14 03:17:09 (12.8 MB/s) - ‘ADSSession2Task2Data.csv’ saved [279369/279369]



Apply the chunk and MapReduce method to sum up each type's value in the synthetic dataset—only keep records which $Select$ value is 1, and read 1000 lines in each chunk. Use multiprocessing package to deploy the task to multi-cores. The output should only have two columns: type and total value.

Hint: filter function is not available in multiprocessing, implement an if-else function in the map or reduce step.

In [14]:
import multiprocessing

In [15]:
df = pd.read_csv('ADSSession2Task2Data.csv')
df[:3]

,Type,Value,Select
0,0,0.1488486295446153,0
1,3,0.2198529412734188,1
2,7,0.15871379686112896,0


In [16]:
print(df.dtypes)


Type      object
Value     object
Select    object
dtype: object


In [17]:
df.describe()

,Type,Value,Select
count,12002,12002,12002
unique,11,10001,3
top,1,0.1488486295446153,1
freq,1269,3,6022


In [18]:
def returnInfo1(record): 
    if record['Select'] =='1':
      return(record['Type'],record['Value'],record['Select'])
    else:
        return (0,0,0)

def typelist(result, record):
    Type, Value = record[0], float(record[1])
    result[Type] = result.get(Type, []) + [Value]
    return result

def sumvalue(record):
    Type, Value = record[0], record[1]
    return [Type, "{:.2f}".format(sum(Value))]

In [19]:
with open('ADSSession2Task2Data.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    chunk = []
    result = []
    # read 100000 rows in each chunk
    chunksize = 1000
    pool = Pool(8)
    for i, line in enumerate(reader):
        
        if (i % chunksize == 0 and i > 0):
            # MapReduce function for rows in one chunk
            output = list(pool.map(sumvalue,list(reduce(typelist,list(pool.map(returnInfo1,reader)),{}).items())))

            result += output
            chunk = []

        chunk.append(line)
    pool.close()
result[1:]

[['3', '269.21'],
 ['0', '279.80'],
 ['1', '287.15'],
 ['9', '272.88'],
 ['6', '276.90'],
 ['7', '260.83'],
 ['8', '250.29'],
 ['5', '273.72'],
 ['4', '294.83'],
 ['2', '265.97']]

In [20]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['Type','Value']).to_csv('output2.csv',index=False)

grading(output)

In [21]:
output[1:]

[['3', '269.21'],
 ['0', '279.80'],
 ['1', '287.15'],
 ['9', '272.88'],
 ['6', '276.90'],
 ['7', '260.83'],
 ['8', '250.29'],
 ['5', '273.72'],
 ['4', '294.83'],
 ['2', '265.97']]

In [22]:
import pandas
import numpy as np
result1 = pd.DataFrame(output, columns=['Type','Valuesum'])
result1 = result1[1:]
result1

,Type,Valuesum
1,3,269.21
2,0,279.80
3,1,287.15
4,9,272.88
5,6,276.90
6,7,260.83
7,8,250.29
8,5,273.72
9,4,294.83
10,2,265.97
